# Homework 5A

## Problem 1

20NG

In [5]:
from nbimporter import NotebookLoader
import numpy as np
loader = NotebookLoader()
load_20ng_data = loader.load_module("HW1").load_20ng_data
tf_normalization = loader.load_module("HW1").tf_normalization

# load data
ng_data, ng_label, ng_category = load_20ng_data('Data/20_newsgroups')

ng_data, ng_label, ng_category = load_20ng_data('Data/20_newsgroups')
ng_tf, ng_features = tf_normalization(ng_data)
ng_label = np.array(ng_label)
#label_encoder = LabelEncoder()
#ng_label_numeric = label_encoder.fit_transform(ng_label)


In [11]:
import numpy as np
from collections import Counter
from sklearn.decomposition import NMF

def get_dominant_category(doc_indices, labels, categories):
    doc_labels = [labels[i] for i in doc_indices]
    if len(doc_labels) == 0:
        return "N/A"
    dominant = Counter(doc_labels).most_common(1)[0][0]
    if isinstance(dominant, int):
        return categories[dominant]
    return dominant

Ks = [10, 20, 50]

print("=== NMF ===\n")

for k in Ks:
    print(f"--- NMF with K={k} ---\n")
    nmf_model = NMF(n_components=k, random_state=42)
    # W: document-topic matrix, H: topic-term matrix
    W = nmf_model.fit_transform(ng_tf)
    H = nmf_model.components_
    
    dominant_assignments = np.argmax(W, axis=1)
    
    for topic_idx, topic in enumerate(H):
        doc_indices = np.where(dominant_assignments == topic_idx)[0]
        dominant_category = get_dominant_category(doc_indices, ng_label, ng_category)
        print(f"Topic {topic_idx} - {dominant_category}):")
        
        # top 20 words for thi topic
        top_indices = topic.argsort()[::-1][:20]
        for idx in top_indices:
            print(f"  {ng_features[idx]}: {topic[idx]:.4f}")

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
ng_counts = vectorizer.fit_transform(ng_data)
feature_names = vectorizer.get_feature_names_out()

print("=== LDA ===\n")
for k in Ks:
    print(f"--- LDA with K={k} ---\n")
    lda_model = LatentDirichletAllocation(n_components=k, random_state=42)
    lda_model.fit(ng_counts)
    
    doc_topic_dist = lda_model.transform(ng_counts)
    dominant_assignments = np.argmax(doc_topic_dist, axis=1)
    
    for topic_idx, topic in enumerate(lda_model.components_):
        doc_indices = np.where(dominant_assignments == topic_idx)[0]
        dominant_category = get_dominant_category(doc_indices, ng_label, ng_category)
        print(f"Topic {topic_idx} (Dominant Category: {dominant_category}):")

        topic_prob = topic / topic.sum()
        top_indices = topic_prob.argsort()[::-1][:20]
        for idx in top_indices:
            print(f"  {feature_names[idx]}: {topic_prob[idx]:.4f}")


=== NMF ===

--- NMF with K=10 ---

Topic 0 (Category: rec.sport.baseball):
  edu: 2.7648
  news: 0.4584
  university: 0.1989
  article: 0.1684
  usenet: 0.1620
  cc: 0.1607
  writes: 0.1596
  references: 0.1538
  gmt: 0.1444
  cs: 0.1416
  uiuc: 0.1370
  nntp: 0.1364
  organization: 0.1343
  apr: 0.1309
  subject: 0.1280
  lines: 0.1259
  message: 0.1259
  path: 0.1255
  date: 0.1249
  id: 0.1247
Topic 1 (Category: rec.motorcycles):
  com: 1.8835
  hp: 0.2066
  writes: 0.1538
  references: 0.1385
  article: 0.1352
  netcom: 0.1243
  sun: 0.1056
  ibm: 0.1029
  news: 0.1018
  gmt: 0.0985
  srv: 0.0983
  apr: 0.0956
  cmu: 0.0931
  message: 0.0903
  id: 0.0870
  organization: 0.0837
  date: 0.0835
  apple: 0.0830
  newsgroups: 0.0827
  path: 0.0826
Topic 2 (Category: comp.os.ms-windows.misc):
  comp: 1.0987
  windows: 0.7517
  os: 0.3960
  ms: 0.3692
  edu: 0.3140
  misc: 0.3046
  sys: 0.2484
  cs: 0.2332
  srv: 0.2028
  cantaloupe: 0.2020
  cmu: 0.1822
  pc: 0.1636
  hardware: 0.1546
 

/opt/anaconda3/envs/PRO312/lib/python3.12/site-packages/sklearn/decomposition/_nmf.py:1742: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


Topic 0 (Category: rec.sport.baseball):
  edu: 3.0173
  university: 0.2281
  nntp: 0.1899
  cc: 0.1709
  posting: 0.1673
  host: 0.1672
  references: 0.1598
  article: 0.1411
  usenet: 0.1348
  organization: 0.1292
  apr: 0.1289
  gmt: 0.1239
  subject: 0.1215
  path: 0.1206
  lines: 0.1198
  id: 0.1185
  date: 0.1184
  newsgroups: 0.1177
  message: 0.1155
  writes: 0.1114
Topic 1 (Category: rec.motorcycles):
  com: 2.0173
  hp: 0.2267
  references: 0.1342
  netcom: 0.1237
  writes: 0.1160
  sun: 0.1122
  article: 0.1039
  gmt: 0.1000
  srv: 0.0953
  apr: 0.0939
  ibm: 0.0877
  message: 0.0872
  id: 0.0869
  organization: 0.0830
  path: 0.0827
  date: 0.0825
  newsgroups: 0.0822
  subject: 0.0799
  lines: 0.0781
  cantaloupe: 0.0778
Topic 2 (Category: comp.windows.x):
  net: 1.6495
  ans: 0.6119
  reston: 0.6048
  howland: 0.6040
  near: 0.4972
  noc: 0.4868
  harvard: 0.3914
  das: 0.3778
  lines: 0.3134
  subject: 0.3131
  message: 0.3118
  id: 0.3095
  date: 0.3082
  newsgroups: 0.3

/opt/anaconda3/envs/PRO312/lib/python3.12/site-packages/sklearn/decomposition/_nmf.py:1742: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


Topic 0 (Category: rec.sport.baseball):
  edu: 3.4012
  university: 0.1989
  references: 0.1390
  article: 0.0843
  writes: 0.0786
  uwm: 0.0434
  umd: 0.0269
  college: 0.0258
  zaphod: 0.0243
  upenn: 0.0231
  colorado: 0.0217
  netnews: 0.0209
  mps: 0.0201
  bu: 0.0201
  david: 0.0194
  ksu: 0.0185
  wisc: 0.0167
  baseball: 0.0160
  michael: 0.0155
  ucsd: 0.0149
Topic 1 (Category: talk.politics.guns):
  com: 2.2517
  edu: 0.2644
  writes: 0.1547
  article: 0.1389
  references: 0.1355
  stratus: 0.0659
  uunet: 0.0638
  mot: 0.0509
  dec: 0.0489
  cmu: 0.0465
  portal: 0.0393
  tek: 0.0392
  ti: 0.0391
  systems: 0.0384
  srv: 0.0373
  id: 0.0365
  message: 0.0362
  corporation: 0.0359
  path: 0.0353
  newsgroups: 0.0351
Topic 2 (Category: comp.graphics):
  net: 2.1500
  ans: 1.0457
  howland: 1.0390
  reston: 1.0350
  edu: 0.4337
  near: 0.2403
  noc: 0.2319
  sura: 0.1759
  1993: 0.0956
  bogus: 0.0949
  zaphod: 0.0664
  newsserver: 0.0628
  darwin: 0.0619
  jvnc: 0.0603
  23: 0

In [27]:
import os
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

def load_duc2001_data_abstract_intro(data_dir):
    abstracts = []
    introductions = []
    count = 0
    
    for filename in os.listdir(data_dir):
        if filename.endswith(".txt"):
            file_path = os.path.join(data_dir, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()
                count += 1
                if "Abstract:" in text and "Introduction:" in text:

                    parts = text.split("Introduction:")
                    abstract_section = parts[0]
                    abstract = abstract_section.split("Abstract:")[-1].strip()
                    introduction = parts[1].strip()
                    
                    abstracts.append(abstract)
                    introductions.append(introduction)
                else:
                    print(f"File {filename} missing Abstract or Introduction section.")
                    
    print(f'{count} documents retrieved!')
    return abstracts, introductions

abstracts, introductions = load_duc2001_data_abstract_intro('Data/DUC2001/Summaries')
#print("First abstract:", abstracts[0])
#print("First introduction:", introductions[0])

Ks = [10, 20, 50]

print("=== NMF Topics on DUC 2001 ===\n")

tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
duc_tf = tfidf_vectorizer.fit_transform(introductions)
duc_features = tfidf_vectorizer.get_feature_names_out()

for k in Ks:
    print(f"--- NMF with K={k} ---\n")
    nmf_model = NMF(n_components=k, random_state=42)
    W = nmf_model.fit_transform(duc_tf) 
    H = nmf_model.components_           
    
    for topic_idx, topic in enumerate(H):
        print(f"Topic {topic_idx}:")
        top_indices = topic.argsort()[::-1][:20]
        for idx in top_indices:
            print(f"  {duc_features[idx]}: {topic[idx]:.4f}")
        print()
    print("="*60, "\n")

print("=== LDA Topics on DUC 2001 ===\n")

count_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
duc_counts = count_vectorizer.fit_transform(introductions)
duc_feature_names = count_vectorizer.get_feature_names_out()

for k in Ks:
    print(f"--- LDA with K={k} ---\n")
    lda_model = LatentDirichletAllocation(n_components=k, random_state=42)
    lda_model.fit(duc_counts)
    
    for topic_idx, topic in enumerate(lda_model.components_):
        print(f"Topic {topic_idx}:")
        topic_prob = topic / topic.sum()
        top_indices = topic_prob.argsort()[::-1][:20]
        for idx in top_indices:
            print(f"  {duc_feature_names[idx]}: {topic_prob[idx]:.4f}")
        print()
    print("="*60, "\n")

File ap900322-0200_system.txt missing Abstract or Introduction section.
304 documents retrieved!
=== NMF Topics on DUC 2001 ===

--- NMF with K=10 ---

Topic 0:
  welfare: 0.9821
  nafta: 0.7547
  diamond: 0.6797
  mr: 0.6394
  beers: 0.5480
  diamonds: 0.4811
  bank: 0.4770
  tunnel: 0.3296
  reform: 0.3293
  says: 0.2885
  countries: 0.2839
  cent: 0.2825
  clinton: 0.2811
  world: 0.2797
  billion: 0.2611
  poverty: 0.2552
  president: 0.2478
  trade: 0.2407
  market: 0.2231
  year: 0.2219

Topic 1:
  oil: 0.7679
  exxon: 0.6930
  spill: 0.5117
  valdez: 0.4435
  cleanup: 0.2237
  alaska: 0.2198
  said: 0.2198
  tanker: 0.1999
  sound: 0.1524
  ship: 0.1358
  guard: 0.1256
  coast: 0.1157
  million: 0.1119
  miles: 0.1091
  reef: 0.1079
  gallons: 0.1038
  prince: 0.1029
  crude: 0.1021
  vessel: 0.0991
  hazelwood: 0.0990

Topic 2:
  hurricane: 0.9195
  hurricanes: 0.3667
  sheets: 0.2631
  storm: 0.2477
  storms: 0.2247
  atlantic: 0.2020
  mph: 0.2017
  winds: 0.1873
  tropical: 

## Problem 2

In [32]:
import os
import re
import math
import numpy as np
from collections import Counter
import subprocess
import nltk

# Download NLTK resource if needed.
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

##############################################
# Loader for DUC Documents (extracting Abstracts)
##############################################
def load_duc_abstracts(data_dir):
    """
    Loads text files from the given directory and extracts the "Abstract:" part.
    It assumes that in each file, the Abstract section starts with "Abstract:" 
    and continues until either "Introduction:" or end-of-file.
    
    Returns a list of abstract texts.
    """
    abstracts = []
    for filename in sorted(os.listdir(data_dir)):
        if filename.lower().endswith(".txt"):
            path = os.path.join(data_dir, filename)
            with open(path, "r", encoding="utf-8") as f:
                text = f.read().strip()
                if text:
                    # Use regex to extract text after "Abstract:" up to "Introduction:" (if present)
                    m = re.search(r'Abstract:\s*(.*?)(?:Introduction:|$)', text, re.DOTALL | re.IGNORECASE)
                    if m:
                        abstract = m.group(1).strip()
                        abstracts.append(abstract)
                    else:
                        print(f"File {filename} missing Abstract section.")
    print("Loaded", len(abstracts), "abstracts from", data_dir)
    return abstracts

##############################################
# KL-Sum Summarizer Class
##############################################
class KLSumSummarizer:
    def __init__(self, documents, mode="word", word_limit=250, epsilon=1e-12, topic_model_params=None):
        """
        documents: List of document strings.
          For DUC evaluation, use the Abstracts (or another appropriate subset).
        mode: "word" for word-based KL-Sum or "topic" for topic-based KL-Sum.
        word_limit: Target summary length (in words).
        epsilon: Smoothing constant.
        topic_model_params: Dictionary of LDA parameters (e.g., {"n_topics":30, "max_iter":50}).
                           Used only if mode=="topic". The key "n_topics" is used to set n_components.
        """
        self.documents = documents
        self.mode = mode
        self.word_limit = word_limit
        self.epsilon = epsilon

        # Combine all documents into one large cluster.
        self.cluster_text = " ".join(documents)
        # Segment the cluster text into sentences.
        self.sentences = sent_tokenize(self.cluster_text)
        print("Total sentences in cluster:", len(self.sentences))
        
        if self.mode == "word":
            # Compute document-level word distribution PD.
            words = self.cluster_text.lower().split()
            pd_counts = Counter(words)
            total_pd = sum(pd_counts.values())
            self.PD = {w: count / total_pd for w, count in pd_counts.items()}
            self.vocab = set(self.PD.keys())
            print("Computed word distribution (PD) over", len(self.PD), "unique words.")
        elif self.mode == "topic":
            # Train an LDA model over the documents.
            self.vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
            doc_counts = self.vectorizer.fit_transform(documents)
            # Process LDA parameters.
            lda_params = topic_model_params.copy() if topic_model_params is not None else {}
            n_topics = lda_params.pop("n_topics", 10)  # Use provided n_topics (default=10)
            lda_params.setdefault("n_components", n_topics)
            lda_params.setdefault("random_state", 42)
            self.lda_model = LatentDirichletAllocation(**lda_params)
            self.lda_model.fit(doc_counts)
            # Compute PD_topic: average topic distribution over all documents.
            doc_topic = self.lda_model.transform(doc_counts)
            self.PD_topic = np.mean(doc_topic, axis=0)
            print("Trained LDA with", n_topics, "topics. PD_topic computed.")
        else:
            raise ValueError("mode must be either 'word' or 'topic'")
        
        # Precompute word counts for each sentence.
        self.sentence_word_counts = []
        for sent in self.sentences:
            counts = Counter(sent.lower().split())
            self.sentence_word_counts.append(counts)
        print("Computed word counts for", len(self.sentence_word_counts), "sentences.")
    
    def total_words(self, counts):
        """Return total word count from a Counter (or dictionary) of counts."""
        return sum(counts.values())
    
    def kl_divergence(self, PD, PS):
        """
        Compute KL divergence KL(PD || PS) = sum_w PD(w) * log(PD(w)/(PS(w)+epsilon)).
        For word mode, PD and PS are dictionaries;
        for topic mode, they are numpy arrays.
        """
        kl = 0.0
        if isinstance(PD, dict):
            for w, p in PD.items():
                q = PS.get(w, 0)
                kl += p * math.log((p / (q + self.epsilon)) + self.epsilon)
        else:
            for i in range(len(PD)):
                p = PD[i]
                q = PS[i]
                if p > 0:
                    kl += p * math.log(p / (q + self.epsilon) + self.epsilon)
        return kl

    def summarize(self):
        """
        Greedily builds a summary by selecting sentences that lower KL(PD || PS).
        For word mode, PS is built from the candidate summary's word counts.
        For topic mode, PS is inferred (via LDA) from the candidate summary (treated as a new document).
        Returns the final summary as a string.
        """
        selected_sentence_indices = []
        if self.mode == "word":
            selected_word_counts = Counter()
        current_summary_length = 0
        candidate_indices = set(range(len(self.sentences)))
        current_KL = float('inf')
        
        while current_summary_length < self.word_limit:
            best_kl = float('inf')
            best_idx = None
            best_state = None  # either new word counts or new topic vector, depending on mode
            
            for idx in candidate_indices:
                if self.mode == "word":
                    sim_counts = selected_word_counts.copy()
                    sim_counts.update(self.sentence_word_counts[idx])
                    new_total = self.total_words(sim_counts)
                    PS = {w: cnt / new_total for w, cnt in sim_counts.items()}
                    kl = self.kl_divergence(self.PD, PS)
                    candidate_state = sim_counts
                else:  # topic mode
                    # Build candidate summary by concatenating selected sentences plus candidate sentence.
                    candidate_text = " ".join([self.sentences[i] for i in selected_sentence_indices + [idx]])
                    vec = self.vectorizer.transform([candidate_text])
                    ps_topic = self.lda_model.transform(vec)[0]
                    # Normalize (with smoothing) the topic distribution.
                    n_topics = len(ps_topic)
                    ps_topic = (ps_topic + self.epsilon) / (np.sum(ps_topic) + self.epsilon * n_topics)
                    kl = self.kl_divergence(self.PD_topic, ps_topic)
                    candidate_state = None  # not stored for topic mode
                
                if kl < best_kl:
                    best_kl = kl
                    best_idx = idx
                    best_state = candidate_state
            
            if best_idx is None or best_kl >= current_KL:
                break
            
            selected_sentence_indices.append(best_idx)
            candidate_indices.remove(best_idx)
            if self.mode == "word":
                selected_word_counts = best_state
            current_summary_length = self.total_words(selected_word_counts) if self.mode == "word" else len(" ".join([self.sentences[i] for i in selected_sentence_indices]).split())
            current_KL = best_kl
            print(f"Added sentence {best_idx} | Summary length: {current_summary_length} words | KL: {best_kl:.4f}")
            
            if current_summary_length >= self.word_limit:
                break
        
        selected_sentence_indices.sort()
        summary = "\n".join([self.sentences[i] for i in selected_sentence_indices])
        return summary

    def evaluate_rouge(self, summary, gold_summary_folder, rouge_config_xml):
        """
        Evaluate the generated summary using the ROUGE Perl package.
        gold_summary_folder: Directory containing human gold summaries (e.g., Abstracts).
        rouge_config_xml: Path to a ROUGE XML configuration file.
        Returns the raw output from the ROUGE evaluation.
        """
        candidate_file = "temp_candidate.txt"
        with open(candidate_file, "w", encoding="utf-8") as f:
            f.write(summary)
        rouge_cmd = [
            "perl", "ROUGE-1.5.5.pl", "-a", "-m", "-n", "2", "-x", "-2", "4", "-u",
            "-c", "95", "-r", "1000", "-f", "A", "-p", "0.5", "-t", "0", rouge_config_xml
        ]
        result = subprocess.run(rouge_cmd, capture_output=True, text=True)
        return result.stdout

##############################################
# Example Usage:
##############################################

# Load documents from DUC Summaries folder (using the Abstract section).
abstracts = load_duc_abstracts("Data/DUC2001/Summaries")

# --- Word-based KL-Sum ---
print("\n=== Word-based KL-Sum Summarization ===")
word_summarizer = KLSumSummarizer(abstracts, mode="word", word_limit=250)
word_summary = word_summarizer.summarize()
print("\n--- Final Word-based Summary ---")
print(word_summary)

# --- Topic-based KL-Sum ---
print("\n=== Topic-based KL-Sum Summarization ===")
# Example LDA parameters: using 30 topics and 50 iterations.
lda_params = {"n_topics": 30, "max_iter": 50}
topic_summarizer = KLSumSummarizer(abstracts, mode="topic", word_limit=250, topic_model_params=lda_params)
topic_summary = topic_summarizer.summarize()
print("\n--- Final Topic-based Summary ---")
print(topic_summary)

# --- ROUGE Evaluation ---
# (Make sure you have a valid ROUGE configuration XML file and that the ROUGE Perl package is installed)
# For example:
rouge_results = word_summarizer.evaluate_rouge(word_summary, gold_summary_folder="Data/Summaries", rouge_config_xml="rouge_config.xml")
print("\n=== ROUGE Evaluation Results ===")
print(rouge_results)

[nltk_data] Downloading package punkt_tab to /Users/OSX/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


File ap900322-0200_system.txt missing Abstract section.
Loaded 303 abstracts from Data/DUC2001/Summaries

=== Word-based KL-Sum Summarization ===
Total sentences in cluster: 1602
Computed word distribution (PD) over 7814 unique words.
Computed word counts for 1602 sentences.
Added sentence 1531 | Summary length: 33 words | KL: 14.4875
Added sentence 1359 | Summary length: 87 words | KL: 13.5694
Added sentence 787 | Summary length: 117 words | KL: 12.8366
Added sentence 375 | Summary length: 146 words | KL: 12.2389
Added sentence 970 | Summary length: 184 words | KL: 11.7367
Added sentence 905 | Summary length: 214 words | KL: 11.4315
Added sentence 198 | Summary length: 249 words | KL: 11.1807
Added sentence 50 | Summary length: 281 words | KL: 10.9715

--- Final Word-based Summary ---
The National Hurricane Center study said that the US death toll from a major hurricane could be far worse than previously predicted as they marked the opening of the 1988 storm season.
In recent B-52 cra